In [1]:
import numpy as np
import random
import os
import sys

def read_binary(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()

    parsed_lines = []

    for line in lines:
        line = line.strip().replace(" ", "")  # 공백 제거
        n_bits = len(line)

        if n_bits % 32 != 0:
            raise ValueError(f"Invalid bit length in line: {line} (not a multiple of 32)")

        n_complex = n_bits // 32
        complex_vals = []

        for i in range(n_complex):
            # 32bit chunk: [real|imag]
            chunk = line[i*32 : (i+1)*32]
            real_bin = chunk[0:16]
            imag_bin = chunk[16:32]
            complex_vals.append([real_bin, imag_bin])

        if n_complex == 1:
            parsed_lines.append([real_bin, imag_bin])
        else:
            parsed_lines.append(complex_vals)

    return parsed_lines

def binary_to_fp16(binary_array):
    float16_array = []

    # print(len(binary_array), "complex numbers found in the file.")
    for line in binary_array:
        complex_line = []
        # print(len(line[0]))
        if(len(line[0]) == 16):
            # Single complex number case
            real_fp16 = np.uint16(int(line[0], 2)).view(np.float16)
            imag_fp16 = np.uint16(int(line[1], 2)).view(np.float16)
            complex_line.append([real_fp16, imag_fp16])
        else:
            for real, imag in line:
                if(real == "xxxxxxxxxxxxxxxx"):
                    real = "1" * 16
                if(imag == "xxxxxxxxxxxxxxxx"):
                    imag = "1" * 16
                real_fp16 = np.uint16(int(real, 2)).view(np.float16)
                imag_fp16 = np.uint16(int(imag, 2)).view(np.float16)
                complex_line.append([real_fp16, imag_fp16])
        float16_array.append(complex_line)

    return float16_array

def pretty_print_fp16_array_inline(array, label="ARRAY"):
    # 한 줄에 최대 몇 개의 complex 값이 있는지 계산 (for header)
    max_complex_per_line = max(len(line) for line in array)

    # 헤더 생성
    header = f"{'Line':>4} |"
    for i in range(max_complex_per_line):
        header += f" {label}_real  | {label}_imag  |"
    print(header)
    print("-" * len(header))

    # 각 line 출력
    for line_idx, line in enumerate(array):
        line_str = f"{line_idx:4d} |"
        for real, imag in line:
            line_str += f" {real:11.5f} | {imag:11.5f} |"
        print(line_str)



In [3]:
kernel = 'fft_0'
read_binary(kernel + "_input.txt")[0]

['0011101101011001', '1011101101100110']

In [4]:
pretty_print_fp16_array_inline(binary_to_fp16(read_binary(kernel + "_input.txt")))

Line | ARRAY_real  | ARRAY_imag  |
----------------------------------
   0 |     0.91846 |    -0.92480 |
   1 |    -0.78027 |    -0.36060 |
   2 |     1.63574 |     1.49512 |
   3 |     0.20715 |    -0.11304 |
   4 |     1.52832 |     1.32812 |
   5 |     1.59375 |     1.92871 |
   6 |    -0.25049 |     1.76953 |
   7 |     1.49219 |     0.64355 |
   8 |     0.60889 |     1.91895 |
   9 |     0.27075 |    -0.16992 |
  10 |    -0.78711 |     0.89307 |
  11 |    -0.26538 |     0.42529 |
  12 |     1.98828 |     0.58740 |
  13 |     1.81738 |    -0.65381 |
  14 |    -0.82861 |     0.75244 |
  15 |     0.14111 |    -0.98242 |
  16 |    -0.33862 |     0.76758 |
  17 |     0.99023 |     1.81641 |
  18 |     0.26953 |    -0.36450 |
  19 |     0.32642 |    -0.35889 |
  20 |     0.68408 |    -0.21179 |
  21 |    -0.33276 |     1.45020 |
  22 |     0.13696 |     1.95605 |
  23 |     0.62061 |    -0.58496 |
  24 |     0.90723 |     0.09448 |
  25 |    -0.55566 |    -0.61768 |
  26 |    -0.70898 |

In [5]:
binary_to_fp16(read_binary(kernel + "_output.txt"))

[[[0.1382, -1.285]],
 [[1.699, -0.5645]],
 [[1.843, 1.382]],
 [[1.429, 1.608]],
 [[3.121, 3.258]],
 [[-0.0654, -0.6006]],
 [[1.242, 2.414]],
 [[-1.742, 1.126]],
 [[0.88, 1.749]],
 [[0.3381, 2.09]],
 [[-1.053, 1.318]],
 [[-0.5215, 0.4678]],
 [[3.805, -0.0664]],
 [[0.1709, 1.241]],
 [[-0.6875, -0.23]],
 [[-0.9697, 1.734]],
 [[0.6514, 2.584]],
 [[-1.329, -1.049]],
 [[0.5957, -0.7236]],
 [[-0.0569, -0.005615]],
 [[0.3513, 1.238]],
 [[1.017, -1.662]],
 [[0.758, 1.371]],
 [[-0.4836, 2.541]],
 [[0.3516, -0.5234]],
 [[1.463, 0.712]],
 [[-1.182, 2.168]],
 [[-0.2366, -1.581]],
 [[-0.4932, 1.3125]],
 [[1.497, -2.24]],
 [[0.4705, 0.2461]],
 [[0.04468, 1.256]],
 [[1.767, 0.1997]],
 [[-1.235, -2.021]],
 [[-0.8896, 1.926]],
 [[-0.6953, 0.638]],
 [[3.035, 1.536]],
 [[0.2227, -1.647]],
 [[3.205, 2.344]],
 [[0.6094, 1.215]],
 [[1.396, -1.834]],
 [[0.83, 0.1089]],
 [[0.126, -0.871]],
 [[-0.8403, -0.335]],
 [[0.986, 0.864]],
 [[-1.385, 0.982]],
 [[1.424, 1.835]],
 [[0.147, 0.2158]],
 [[3.025, -0.3794]],
 

In [6]:
pretty_print_fp16_array_inline(binary_to_fp16(read_binary(kernel + "_weight.txt")))

Line | ARRAY_real  | ARRAY_imag  |
----------------------------------
   0 |     1.00000 |    -0.00000 |


In [7]:
pretty_print_fp16_array_inline(binary_to_fp16(read_binary(kernel + "_out.txt")))

Line | ARRAY_real  | ARRAY_imag  | ARRAY_real  | ARRAY_imag  | ARRAY_real  | ARRAY_imag  | ARRAY_real  | ARRAY_imag  | ARRAY_real  | ARRAY_imag  | ARRAY_real  | ARRAY_imag  | ARRAY_real  | ARRAY_imag  | ARRAY_real  | ARRAY_imag  | ARRAY_real  | ARRAY_imag  | ARRAY_real  | ARRAY_imag  | ARRAY_real  | ARRAY_imag  |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
   0 |     0.00000 |     0.00000 |     0.00000 |     0.00000 |     0.00000 |     0.00000 |     0.00000 |     0.00000 |     0.00000 |     0.00000 |     0.00000 |     0.00000 |     0.00000 |     0.00000 |     0.00000 |     0.00000 |     0.00000 |     0.00000 |     0.00000 |     0.00000 |     0.00000 |     0.00000 |
   1 |     0.00000 |     0.00000 |     0.00000 |     0.

In [23]:
len(result)

158

In [5]:

kernel_list = ['fft_0', 'fft_1', 'fft_2', 'fft_3', 'fft_4', 'fft_5', 'fft_6', 'fft_7']
# kernel = 'fft_0'

for kernel in kernel_list:
    golden = binary_to_fp16(read_binary(kernel + "_output.txt"))
    result = (binary_to_fp16(read_binary("../OUTPUT/kernel + "_out2.txt")))
    j = 0
    if(kernel == 'fft_7'):
        start = 69+64
    elif(kernel == 'fft_6'):
        start = 69
    elif (kernel == 'fft_5'):
        start = 69-32
    elif (kernel == 'fft_4'):
        start = 69-32-16
    elif (kernel == 'fft_3'):
        start = 69-32-16-8
    elif (kernel == 'fft_2'):
        start = 69-32-16-8-4
    elif (kernel == 'fft_1'):
        start = 69-32-16-8-4-2
    elif (kernel == 'fft_0'):
        start = 69-32-16-8-4-2-3
    # print(start)
    error = 0
    print(start)
    for i in range(start, start+256):
        if(result[i][10] != golden[j][0]):
            print(j, result[i][10], golden[j][0], result[i][10] == golden[j][0])
            error =1
            print("-----------------------------------------------WRONG----------------------------------------------------")
        j += 1
    print(f'{kernel} : ', error)

SyntaxError: invalid syntax (4228246817.py, line 6)

In [40]:
for i in range(69, 69+256):
    print(i-69, result[i][10])

0 [-7.523, 4.617]
1 [9.83, -8.07]
2 [-0.596, -7.1]
3 [5.516, 7.53]
4 [8.984, -1.229]
5 [3.0, 2.928]
6 [6.57, 4.68]
7 [6.367, 1.457]
8 [0.8438, -2.672]
9 [8.875, -5.285]
10 [12.18, 2.963]
11 [1.609, 8.22]
12 [-2.898, -10.9]
13 [7.254, 1.527]
14 [-9.99, 6.37]
15 [1.055, -0.4307]
16 [7.387, 4.656]
17 [15.555, 7.67]
18 [0.4492, -5.914]
19 [4.934, 0.668]
20 [-7.04, -14.92]
21 [-3.38, -4.43]
22 [-4.61, -8.13]
23 [12.125, -2.754]
24 [0.6055, 15.375]
25 [7.836, -4.055]
26 [-6.68, 7.5]
27 [-16.3, 2.973]
28 [0.586, -10.79]
29 [-12.7, 0.0664]
30 [5.98, 2.94]
31 [1.23, -8.67]
32 [39.72, 31.27]
33 [1.924, 6.42]
34 [1.026, -2.953]
35 [1.523, -11.48]
36 [1.504, 1.494]
37 [2.906, 5.52]
38 [-0.1416, -2.82]
39 [6.723, 0.416]
40 [-15.6, -9.22]
41 [-0.8584, -7.0]
42 [2.979, -3.467]
43 [-7.094, -0.3906]
44 [17.38, 6.258]
45 [-5.312, 0.4434]
46 [-7.773, -1.713]
47 [4.297, 15.33]
48 [-0.9375, 4.094]
49 [-3.006, 7.98]
50 [-11.75, 4.047]
51 [0.03125, -0.9]
52 [2.223, 0.2559]
53 [-2.41, 0.06836]
54 [0.551, -8.9

In [4]:

kernel_list = ['fft_0', 'fft_1', 'fft_2', 'fft_3', 'fft_4', 'fft_5', 'fft_6', 'fft_7']
# kernel_list = ['fft_3']

for kernel in kernel_list:
    golden = binary_to_fp16(read_binary(kernel + "_output.txt"))
    result = (binary_to_fp16(read_binary(kernel + "_out.txt")))
    j = 0
    if(kernel == 'fft_7'):
        start = 69+64
    elif(kernel == 'fft_6'):
        start = 69
    elif (kernel == 'fft_5'):
        start = 69-32
    elif (kernel == 'fft_4'):
        start = 69-32-16
    elif (kernel == 'fft_3'):
        start = 69-32-16-8
    elif (kernel == 'fft_2'):
        start = 69-32-16-8-4
    elif (kernel == 'fft_1'):
        start = 69-32-16-8-4-2
    elif (kernel == 'fft_0'):
        start = 69-32-16-8-4-2-3
    # print(start)
    error = 0
    for i in range(start, start+256):
        if(result[i][10] != golden[j][0]):
            print(j, result[i][10], golden[j][0], result[i][10] == golden[j][0])
            error =1
            print("-----------------------------------------------WRONG----------------------------------------------------")
        j += 1
    print(f'{kernel} : ', error)

0 [0.0, 0.0] [0.1382, -1.285] False
-----------------------------------------------WRONG----------------------------------------------------
1 [0.0, 0.0] [1.699, -0.5645] False
-----------------------------------------------WRONG----------------------------------------------------
2 [0.0, 0.0] [1.843, 1.382] False
-----------------------------------------------WRONG----------------------------------------------------
3 [0.0, 0.0] [1.429, 1.608] False
-----------------------------------------------WRONG----------------------------------------------------
4 [0.0, 0.0] [3.121, 3.258] False
-----------------------------------------------WRONG----------------------------------------------------
5 [0.0, 0.0] [-0.0654, -0.6006] False
-----------------------------------------------WRONG----------------------------------------------------
6 [0.0, 0.0] [1.242, 2.414] False
-----------------------------------------------WRONG----------------------------------------------------
7 [0.0, 0.0] [-1.74

IndexError: list index out of range